In [5]:
# Read file for upper envelope

def read_file_ue(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        
    Y = []

    for line in lines:
        if line.startswith('\n'): continue
        else:
            arr = line.replace('\n', '').replace('(', '').replace(')', '').split(',')
            Y.append((float(arr[0]), float(arr[1])))

    return Y
    
    
Y = read_file_ue('./instances/ue.txt')
print(Y)

[(1.0, 0.0), (-1.0, 0.0), (0.0, 0.5), (-2.1, 6.0), (-4.7, 0.0)]


In [9]:
n = 100
xk = random.randint(5,20)
x = np.linspace(-xk, xk, num=n)

NameError: name 'random' is not defined

In [7]:
''' 
    Y : set of functions [(a1,b1),(a2,b2),...,(am,bm)]
    x : a vector where functions will be defined 
    plot_line(x, Y) : plot all functions y in Y for a given vector x
'''
def plot_line(x, Y):
    for y in Y:
        plt.plot(x,y[0]*x+y[1],label = (str(y[0])+'x+'+str(y[1])))
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.xlabel("x")
    plt.ylabel("y")

''' 
    x : a vector where functions will be defined 
    uE : list of coordinates of the upper envelope
    plot_dashed(x, uE) : plot the upper envelope with its breakpoints
'''
def plot_dashed(x, uE):
    for item in uE : 
        k = item[0][0]*x+item[0][1]
        i1 = item[1][0] # lower index  
        i2 = item[1][1] # upper index
        t1 = plt.plot(x[i1:i2],k[i1:i2],linewidth=3, dashes=(4, 8), color='red') # plot upperEnvelope
        plt.plot(x[i1],k[i1],'ro', markersize=8) # plot breakpoint
    plt.plot(x[uE[len(uE)-1][1][1]],k[uE[len(uE)-1][1][1]],'ro',markersize=8)
    plt.legend(t1, ['Upper Envelope'])
    
    
# divide Y to single element
def divide(x, Y):
    if len(Y) == 1: return Y[0]
    else : 
        y1 = Y[:len(Y)//2]
        y2 = Y[len(Y)//2:]
        y1 = divide(x,y1)
        y2 = divide(x,y2)
        return conquer(x,y1, y2)


'''
f is an upper enveloppe : list of sublist of 2 tuples
create(x,f) : return a set of point f1 = [f(xi) for i in len(x)]
'''
def create(x, f): 
    f1 = []
    for item in f:
        k = item[0][0]*x[item[1][0]:item[1][1]+1] + item[0][1]
        f1.extend(k)
    return np.array(f1)



'''
f : a function, it can be only coordinate of the functon or a list of sublist
start : lower bound index of the interval studied
end : upper bound index of the interval studied
getFunctionOn(f, start, end) : return coordinate of f on [start, end] as a list of sublist of two tuples
'''
def getFunctionOn(f, start, end):
    I = []
    # if f is a list => upperEnvelope => not necessarily a linear line
    if len(f) != 0 and type(f[0])==list :
        # for each subfunction of f
        for subFunc in f : 
            lowerIndex = subFunc[1][0]
            upperIndex = subFunc[1][1]
            # current subFunc is above on the interval R = [start, end]
            if (lowerIndex <= start and end <= upperIndex):
                I.append([subFunc[0],(start,end)])
                return I
            # current subFunc is above only on [lowerIndex, end]
            elif (start <= lowerIndex and end <= upperIndex and end >= lowerIndex):
                I.append([subFunc[0],(lowerIndex,end)])
                end = lowerIndex - 1
            # current subFunc is above only on [start, upperIndex]
            elif (lowerIndex <= start and upperIndex <= end and start <= upperIndex):
                I.append([subFunc[0],(start,upperIndex)])
                start = upperIndex + 1
            # current subFunc is above only on [lowerIndex, upperIndex]
            #elif (start <= lowerIndex and upperIndex <= end): 
            #    print("4")
            #    I.append([subFunc[0],(lowerIndex, upperIndex)])
            #    I.extend(getFunctionOn(f, start, lowerIndex))
            #    I.extend(getFunctionOn(f, upperIndex, end))
            #   return I
    else : # f is a linear function
        I.append([f,(start,end)])
        return I

'''
f, g : functions (it can only be coordinate of function or function as list of sublists)
x : given vector 
conquer(x, f, g) : return the upper envelope between f and g over x
''' 


def conquer(x, f, g):
    upperE = []
    # calcule [f(xi) for i in range(x)]
    if len(f) != 0 and type(f[0])!=list :
        f1 = f[0]*x + f[1]
    else : f1 = create(x, f)
    # calcule [g(xi) for i in range(x)]
    if len(g) != 0 and type(g[0])!=list :
        g1 = g[0]*x + g[1]
    else : g1 = create(x, g)
    # get the sign of the function f1-g1
    h = np.sign(f1-g1)
    # get index of the points where a change of sign takes place
    idx = np.argwhere(np.diff(h)).flatten() 
    # it means : f>g or f<g or f=g
    if len(idx)==0:
        if -1 in h : # g is above f
            upperE.extend(getFunctionOn(g, 0, len(x)-1))
        else :  # f is above g
            upperE.extend(getFunctionOn(f, 0, len(x)-1))
    else:
        # for all segment between breakpoint :
        # if k breakpoints in idx => k+1 segment
        for i in range(len(idx)+1): 
            if i==0 :
                start, end = i, idx[i]
            elif i == len(idx):
                start, end = idx[i-1]+1, len(h)-1
            else :
                start, end = idx[i-1]+1, idx[i]
            if -1 in h[start:end] : # g is bigger than f
                # add coordinate of g over the [start,end]
                upperE.extend(getFunctionOn(g, start, end))
            else : 
                # add coordinate of f over the [start,end]
                upperE.extend(getFunctionOn(f, start, end))
    return upperE

def findUE_DC(x,Y):
    plot_line(x,Y)
    z = divide(x,Y)
    plot_dashed(x, z)
    plt.show()
    


In [8]:
plot_line(x,Y)
plt.title("Given set of functions")
plt.show()

NameError: name 'x' is not defined